In [146]:
# Import the libraries 
from textblob import TextBlob
import tweepy
import sys
import numpy as np
import csv
import ssl
import pandas as pd
import re

In [147]:
# Twitter API credentials
# Added fake credentials here for privacy
api_key = 'xyz'
api_secret = 'abc'

access_token = '123'
access_secret = '789'

In [148]:
# Create the authentication object
auth_handler = tweepy.OAuthHandler(consumer_key=api_key, consumer_secret=api_secret)
# Set the access token and access token secret
auth_handler.set_access_token(access_token, access_secret)

In [149]:
#Create the API object while passing in the auth information
api = tweepy.API(auth_handler)

In [150]:
#Select tweet whose replies will be extracted
name = 'ManUtd'
tweet_id = '123' #Not an authentic tweet id

In [ ]:
# Extract Replies to the tweet
replies=[]
for tweet in tweepy.Cursor(api.search_tweets,q='to:'+name, result_type='recent').items(1500):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet.text)

In [132]:
# Turn replies into a dataframe to clean
df = pd.DataFrame([tweets for tweets in replies], columns=['Tweets'])
df

,Tweets


In [152]:
#Fuction to clean replies using RegEx
def cleantxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:/\/\S+', '', text)
    return text

In [91]:
#Clean Text
df['Tweets'] = df['Tweets'].apply(cleantxt)
df

,Tweets
0,"We need to see more from our beloved players,..."
1,Poorest
2,Lakaka
3,Ass
4,Bence Ronaldo pedekhos
...,...
106,Off
107,The players are shit and they made Ole sacke...
108,Abysmal
109,The players are lazy it’s not the coach.. the...


In [92]:
#Use TextBlob to get the Polarity of each Tweet
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity


def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

In [98]:
#Calculate the number of positive, neutral and negative tweets
pos = 0
neg = 0
neutral = 0
for i in range(len(df['Tweets'])):
    if df['Polarity'].loc[i] >= 0:
        pos += 1
    elif df['Polarity'].loc[i] == 0:
        neutral += 1
    else:
        neg += 1
        
        
print('pos =', pos)
print('neutral =', neutral)
print('neg =', neg)

pos = 75
neutral = 0
neg = 36


In [156]:
#Net sum of the polarity of replies for this particular match day
summ = df['Polarity'].sum()
summ

4.27

In [154]:
#Polarity adjusted for number of replies
Total_responses = pos + neg
Pol_score = summ + Total_responses
Per = ((Pol_score)/(Total_responses*2))*100
Per

52.03333333333333